## Lab 04

## Zadanie 1 (Igor Sieradzki)

Funkcja <t>flow_rate</i> do modyfikacji

In [32]:
def flow_rate(weight, time, period=1, units_per_kg=1):
    """ Funkcja wylicza ile wagi produktu przybyło/ubyło w jednostce czasu """
    if type(units_per_kg) != int:
        raise TypeError("Parameter 'units_per_kg' has to be an integer")
    
    weight_in_units = weight * units_per_kg
    return (weight_in_units / time) * period

weight = 0.5
time = 3

In [33]:
flow = flow_rate(weight, time)
print("{0:.3} kg per second".format(flow))

0.167 kg per second


In [ ]:
Zmodyfikuj funckje <i>flow_rate</i> tak, aby poniższe wywołania działały poprawnie.

In [26]:
flow = flow_rate(weight, time, period=60, units_per_kg=1000)
print("{} grams per minute".format(flow))

10000.0 grams per minute


In [27]:
flow = flow_rate(weight, time, period=1, units_per_kg=1)
print("{0:.3} kg per second".format(flow))

0.167 kg per second


In [28]:
flow = flow_rate(weight, time)
print("{0:.3} grams per minute".format(flow))

0.167 grams per minute


Wyjaśnić czemu poniższe rzucanie błędu jest porządane

In [31]:
try:
    flow = flow_rate(weight, time, 3600, 2.2)
except TypeError:
    print(True)

True


## Zadanie 2 (Igor Sieradzki)

Dopisz definicje dekoratora _timeit_, który wypisze na wyjscie standardowe czas wywołania udekorowanej funckji np.

```
@timeit
def foo(x):
    return x**2
    
r = foo(2)
```
wypize: <br>
    `Function foo took: 0.00001 seconds` <br>
oraz zwróci 4

In [40]:
from time import time

def timeit(func):
    """ Wypisuje czas wywołania udekorowanej funckji """
    def wrapper(*args, **kwargs):
        t1 = time()
        ret = func(*args, **kwargs)
        t2 = time()
        print("Function {} took {:.5f}s".format(func.__name__, t2-t1))
        return ret
    return wrapper

@timeit
def squares_list(n):
    squares = []
    for i in range(n):
        squares.append(i ** 2)
    return squares

@timeit
def squares_comprehension(n):
    return [i ** 2 for i in range(n)]

@timeit
def squares_map(n):
    return map(lambda x: x**2, range(n))

n = 1000000
l = squares_list(n)
c = squares_comprehension(n)
m = squares_map(n)

Function squares_list took 0.25774s
Function squares_comprehension took 0.20511s
Function squares_map took 0.00000s


## Zadanie 3 (Igor Sieradzki)

Dopisz definicje dekoratora _derivate_ wg. instrukcji w _docstringu_

In [85]:
import sys
sys.float_info.epsilon # epsilon maszynowy


def derivate(epsilon=None):
    """
    Zwraca pochodną funkcji w punkcie, wg. wzoru f'(x) = [f(x+h) - f(x)]/h, 
    gdzie h jest parametrem dekoratora, jeśli nie zostanie podany, należy przyjąć 1000 * epsilon maszynowy
    """
    epsilon = epsilon or 1000 * sys.float_info.epsilon
    
    def wrapper(func):
        def inner(x0):
            return (func(x0+epsilon) - func(x0)) / epsilon
        
        return inner
    
    return wrapper


@derivate(0.01)
def f(x):
    return x*x

@derivate(0.00001)
def g(x):
    return x*x*x+3

def test(a, b, eps=1):
    return abs(round(a)-round(b)) < eps

print(test(f(100), 200.0))
print(round(f(0)) == 0.0)

print(test(g(100), 30000.0))
print(round(g(0)) == 0.0)

from random import random
for x in [random()*1000. for _ in range(20)]:
    print(f(x), 2*x, '\t', test(f(x), 2*x))
    print(g(x), 3*x**2, '\t', test(g(x), 3*x**2))

True
True
True
True
1536.3505534362048 1536.3405534277965 	 True
1770256.7517757413 1770256.7220801208 	 True
542.9715563950595 542.9615563952988 	 True
221105.44629395005 221105.4387924039 	 True
1275.7447130803484 1275.7347130841197 	 True
1220624.3067979813 1220624.293625866 	 True
639.4375078292796 639.4275078306902 	 True
306650.66204965115 306650.6533279255 	 True
253.96765448676888 253.95765448652296 	 True
48370.87152991443 48370.867704222124 	 True
1654.4146086554974 1654.4046086588648 	 True
2052790.9755706785 2052790.9568637689 	 True
1599.960798304528 1599.9507983086783 	 True
1919881.9279670713 1919881.9177564327 	 True
475.21895665777265 475.20895665794274 	 True
169367.67157167196 169367.66436594786 	 True
1252.0759886829183 1252.065988683869 	 True
1175751.9453763962 1175751.9300141856 	 True
135.4582490366738 135.4482490365716 	 True
13759.673165623097 13759.671125304842 	 True
701.0703976047807 701.0603976055801 	 True
368614.2705380916 368614.2608181705 	 True
1439.5

## Zadanie 4 (Igor Sieradzki)

Dopisz definicje dekoratora _accepts_ wg. instrukcji w _docstringu_

In [80]:
def accepts(*types):
    """Sprawdza czy udekorowanej funkcji zostały podane odpowiednie parametry zdefiniowane 
       w argumentach dekoratora"""
    def wrapper(f):
        def inner(*args, **kwargs):
            for arg in [*args, *kwargs.values()]:
                if type(arg) not in types:
                    raise TypeError(f"Type {type(arg)} of argument {arg} is not in a list of " +
                                    f"allowed types for function {f.__name__}. The allowed types are: {types}.")
            else:
                return f(*args, **kwargs)
        
        return inner
    
    return wrapper


@accepts(str)
def capitalize(word):
    return word[0].upper() + word[1:]

print(capitalize('ola') == 'Ola')

try:
    capitalize(2)
except TypeError:
    print(True)

@accepts(float, int)
def static_pow(base, exp):
    return base ** exp 

print(static_pow(2., 2) == 4.)
print(static_pow(2., exp=2) == 4.)
print(static_pow(base=2., exp=2) == 4.)

try:
    static_pow('x', 10)
except TypeError:
    print(True)
    
try:
    static_pow(2, 2.2)
except TypeError:
    print(True)

True
True
True
True
True
True


## Zadanie 5 (Igor Sieradzki)

Dopisz definicje dekoratora _returns_ wg. instrukcji w _docstringu_

In [102]:
def returns(*types):
    """Sprawdza czy udekorowana funkcja zwraca poprawne argumenty, zdefiniowane w parametrach dekoratora"""
    def wrapper(f):
        def inner(*args, **kwargs):
            ret = f(*args, **kwargs)
            if not {type(value) for value in ret}.issubset(types):
                raise TypeError
            else:
                return ret
        
        return inner

    return wrapper


@returns(str)
def str_only_identity(word):
    return word

print(str_only_identity('hello') == 'hello')

try:
    str_only_identity(10)
except TypeError:
    print(True)
    
@returns(int, int)
def split_indices(x):
    return x[0], x[1]

print(split_indices(x=[6,9]) == (6,9))

try:
    split_indices('AB')
except TypeError:
    print(True)

True
True
True
True


## Zadanie 6 (Igor Sieradzki)
Stwórz dekorator cached służący do cachowania wywołań dowolnej funkcji, tzn. chcemy by:
* wywołanie funkcji z określonymi argumentami miało miejsce tylko raz
* funkcja mogła przyjmować dowolną liczbę nazwanych i nienazwanych argumentów
* nie musi reagować poprawnie na domyślne argumenty, tzn. wywołanie funkcji z domyślnymi argumentami a podanie dokładnie takich samych może być traktowane jako dwa różne wywołania
* na opakowanej funkcji można wywołać `.cache_reset()`, który usunie cache z pamięci
* wywołanie `.cache_status()` zwraca string z opisem w postaci: <br>
    `Function FUNCTION_NAME called X times, evaluated Y times`

In [103]:
from random import random

class Cached:
    def __init__(self, f):
        self.cache = {}
        self.count_called = 0
        self.count_eval = 0
        self.f = f
    
    def __call__(self, *args, **kwargs):
        key = ((*args,), (*sorted(kwargs.items(), key=lambda x: x[0]),))
        if key not in self.cache:
            self.count_eval += 1
            self.cache[key] = self.f(*args, **kwargs)
        self.count_called += 1
        return self.cache[key]
    
    def cache_reset(self):
        self.cache = {}
    
    def cache_status(self):
        return f'Function {self.f.__name__} called {self.count_called} times, evaluated {self.count_eval} times'

@Cached
def foo(x, y=1, z=4):
    return random()
        
print(foo(3) == foo(3))
print(foo(4) == foo(4))
print(foo(3, z=-1, y=3) == foo(3, y=3, z=-1))
print(foo(3) != foo(x=3))
a = foo(3)
foo.cache_reset()
print(a != foo(3))
print(foo.cache_status() == 'Function foo called 10 times, evaluated 5 times')

True
True
True
True
True
True


## Zadanie 7 (Krzysztof Hajto)

Napisz dekorator który będzie robić n-krotne złożenie funkcji, gdzie n jest parametrem dekoratora

In [105]:
def zlozenie(n):
    def wrapper(f):
        def inner(*args, **kwargs):
            res = f(*args, **kwargs)
            for i in range(n-1):
                res = f(res)
            return res
        return inner
    return wrapper

@zlozenie(3)
def f1(x):
    return x+1

@zlozenie(2)
def f2(x):
    return x*x

@zlozenie(5)
def f3(word):
    return "".join(chr(ord(l)+1) for l in word)

print(f1(2)==5)
print(f2(3)==81)
print(f3("alamakota")=="fqfrfptyf")

True
True
True


## Zadanie 8 (Krzysztof Hajto)

Python nie ma wbudowanej instrukcji switch. Ale posiada anonimowe funkcje oraz słowniki. Zaimplementuj poniższy switch w postaci słownika funkcji.
`
int my_function(x, y) {


    switch(x) {
        case 1: return y*y;
        case 2: return x+y;
        case 3: return x*y;
        case 4: return 0;
    }
}
`

PS. Nigdy nie róbcie tego w faktycznym kodzie :)

In [107]:
def my_function(x, y):
    return {1: lambda x: x**2,
            2: lambda x: x+2,
            3: lambda x: x*3,
            4: lambda x: 0}[x](y)

print(my_function(1,3)==9)
print(my_function(2,4)==6)
print(my_function(3,1)==3)
print(my_function(4,9)==0)

True
True
True
True
